In [1]:
import tensorflow as tf
from gene_expression import *
from pathway_hierarchy import *
from utils import *
import torch

2024-07-10 14:23:30.706704: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-10 14:23:30.716012: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-10 14:23:30.727919: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-10 14:23:30.727937: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-10 14:23:30.737158: I tensorflow/core/platform/cpu_feature_gua

In [2]:
print(tf.config.list_physical_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


2024-07-10 14:23:47.073582: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-10 14:23:47.074264: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-10 14:23:47.074369: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [3]:
import pandas as pd
import numpy as np

In [4]:
tf.test.is_gpu_available()

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


2024-07-10 14:23:49.794936: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-10 14:23:49.795110: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-10 14:23:49.795206: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

True

In [5]:
torch.cuda.is_available()


True

In [ ]:
rdata = pd.read_csv('../../usman/Single_Cell_Microglia_Project/preprocessed_data/inhibitory_neuron/train_test_set/train_count_matrix.csv', index_col=0)
qdata = pd.read_csv('../../usman/Single_Cell_Microglia_Project/preprocessed_data/inhibitory_neuron/train_test_set/test_count_matrix.csv', index_col=0)
rlabel = pd.read_csv('../../usman/Single_Cell_Microglia_Project/preprocessed_data/inhibitory_neuron/train_test_set/train_label.csv')
qlabel = pd.read_csv('../../usman/Single_Cell_Microglia_Project/preprocessed_data/inhibitory_neuron/train_test_set/test_label.csv')

In [ ]:
single_cell_metadata = pd.read_csv('../../usman/Single_Cell_Microglia_Project/preprocessed_data/inhibitory_neuron/metadata_inhibitory_neurons.csv')

In [ ]:
single_cell_metadata

In [ ]:
# Verifying Train Label is correct

list(single_cell_metadata.set_index('cell_id').loc[rdata.columns,].clinical_pathological_AD) == list(rlabel.diagnosis)

In [ ]:
# Verifying Test Label is correct

list(single_cell_metadata.set_index('cell_id').loc[qdata.columns,].clinical_pathological_AD) == list(qlabel.diagnosis)

In [ ]:
rdata = rdata.T
qdata = qdata.T
rdata

In [ ]:
rdata['diagnosis'] =  list(rlabel.diagnosis)
qdata['diagnosis'] =  list(qlabel.diagnosis)

In [ ]:
rdata.diagnosis.value_counts()

In [ ]:
rdata_neg = rdata[rdata.diagnosis == 'NCI_with_No_Plaques'].iloc[0:24372,]
rdata_pos = rdata[rdata.diagnosis == 'AD_with_Plaques'].iloc[0:24372,]
rdata = pd.concat([rdata_neg,rdata_pos])


In [ ]:
rdata

In [ ]:
rdata = rdata.sample(frac = 1)

In [ ]:
rdata

In [ ]:
rlabel = pd.DataFrame(rdata.diagnosis)

In [ ]:
rlabel

In [ ]:
rdata = rdata.drop(columns=['diagnosis'])

In [ ]:
qdata.diagnosis.value_counts()

In [ ]:
qdata_neg = qdata[qdata.diagnosis == 'NCI_with_No_Plaques'].iloc[0:3442,]
qdata_pos =qdata[qdata.diagnosis == 'AD_with_Plaques'].iloc[0:3442,]
qdata = pd.concat([qdata_neg,qdata_pos])

In [ ]:
qdata

In [ ]:
qdata = qdata.sample(frac = 1)

In [ ]:
qdata

In [ ]:
qlabel = pd.DataFrame(qdata.diagnosis)

In [ ]:
qlabel.reset_index(drop = True, inplace = True)
rlabel.reset_index(drop = True, inplace = True)
rlabel

In [ ]:
qdata = qdata.drop(columns=['diagnosis'])

In [ ]:
qdata

In [ ]:
rdata = rdata.T
qdata = qdata.T

In [ ]:
r_data_tmp = rdata.copy()
q_data_tmp = qdata.copy()
r_label_tmp = rlabel.copy()

In [ ]:
import numpy as np
import pandas as pd


def fphi(x):
  zt = (1 - (x + 1).div(x.max(axis=1) + 1, axis=0)).sum(axis=1) / (x.shape[1] - 1)
  zp = (x + 1).div(x.max(axis=1) + 1, axis=0).mul(zt, axis=0)
  return zp


def frho(x):
  zt = (1 - (1 / (x + 1)).mul(x.min(axis=1) + 1, axis=0)).sum(axis=1) / (x.shape[1] - 1)
  zp = (1 / (x + 1)).mul(x.min(axis=1) + 1, axis=0).mul(zt, axis=0)
  return zp


def fmkg(phi, rho, thr1=0.95, thr2=0.9):
  #print(phi)
  #print(rho)
  gnm = phi.index.values
  ctpnm = phi.columns.values
  phi = np.array(phi)
  rho = np.array(rho)
  nummkg1 = round((1 - thr1) * phi.shape[0])
  nummkg2 = round((1 - thr2) * phi.shape[0])
  print(nummkg1)
  print(nummkg2)
  alpha = []
  beta = []
  for i in range(0, phi.shape[1]):
    alpha = np.append(alpha, np.quantile(phi[:, i], thr1))
    beta = np.append(beta, np.quantile(rho[:, i], thr2))
  mkh = []
  mkl = []
  for i in range(0, phi.shape[1]):
    mkh = np.concatenate([mkh, gnm[phi[:, i] >= alpha[i]][0:nummkg1]], axis=0)
    mkl = np.concatenate([mkl, gnm[rho[:, i] >= beta[i]][0:nummkg2]], axis=0)
  
  print(len(mkh))
  print(len(mkl))
  mkh = mkh.reshape(nummkg1, phi.shape[1])
  mkl = mkl.reshape(nummkg2, rho.shape[1])
  mkh = pd.DataFrame(mkh, columns=ctpnm)
  mkl = pd.DataFrame(mkl, columns=ctpnm)
  print(mkh)
  print(mkl)
  return mkh, mkl


def get_expression(rdata, qdata, rlabel, thrh=0.95, thrl=0.9, normalization=True, marker=True):
    # calculate sum of cell type
    rulabel = rlabel.iloc[:, 0].unique()
    rdt = pd.DataFrame(data=None, columns=None)
    for l in rulabel:
      rdata_l = rdata.iloc[:, rlabel[(rlabel["diagnosis"] == l)].index.tolist()] #row value in rlabel, is column in rdata
      zs = rdata_l.apply(lambda x: x.sum(), axis=1)
      rdt = pd.concat([rdt, pd.DataFrame(data=zs, columns=[l])], axis=1)
      

    # normalization
    if normalization:
        rdt_df = rdt
        rdata_df = rdata
        qdata_df = qdata
        rdt = np.array(rdt_df, dtype=np.float32)
        rdata = np.array(rdata_df, dtype=np.float32)
        qdata = np.array(qdata_df, dtype=np.float32)
        rdt = np.divide(rdt, np.sum(rdt, axis=0, keepdims=True)) * 10000
        rdata = np.log2(np.divide(rdata, np.sum(rdata, axis=0, keepdims=True)) * 10000 + 1)
        qdata = np.log2(np.divide(qdata, np.sum(qdata, axis=0, keepdims=True)) * 10000 + 1)

        rdt = pd.DataFrame(data=rdt, columns=rdt_df.columns, index=rdt_df.index)
        rdata = pd.DataFrame(data=rdata, columns=rdata_df.columns, index=rdata_df.index)
        qdata = pd.DataFrame(data=qdata, columns=qdata_df.columns, index=qdata_df.index)
    #print(rdt.head())
    #print(rdata.head())
    # match gene ensembl ids between the query data and the reference data
    nm = qdata.index.tolist()
    zg = pd.DataFrame(data=nm, columns=['gene'])
    gid = []
    for i in range(0, len(nm)):
      if nm[i] not in rdata.index.tolist():
        zg.iloc[i] = ''
      else:
        gid = gid + [nm[i]]
    qdata = qdata.loc[gid, :]
    rdata = rdata.loc[gid, :]
    rdt = rdt.loc[gid, :]

    rlabel.index = rdata.columns

    if not marker:
        train_x = rdata
        train_y = rlabel
        test_x = qdata
    else:
        # calculate important genes
        phi = fphi(rdt)
        rho = frho(rdt)
        mkh, mkl = fmkg(phi, rho, thrh, thrl)
        mkg = np.unique(np.append(np.array(mkh).reshape(1, -1), np.array(mkl).reshape(1, -1)))

        # return training set and testing set
        train_x = rdata.loc[mkg, :]
        train_y = rlabel
        test_x = qdata.loc[mkg, :]
    return train_x, test_x, train_y


In [ ]:

r_data_tmp = rdata.copy()
q_data_tmp = qdata.copy()
r_label_tmp = rlabel.copy()
train_x, test_x, train_y = get_expression(r_data_tmp,
                                              q_data_tmp,
                                              r_label_tmp,
                                              thrh=0.99,
                                              thrl=0.99,
                                              normalization=True,
                                              marker=False)

In [ ]:
train_x

In [ ]:
train_y

In [ ]:
train_y.diagnosis.value_counts()

In [ ]:
test_x

In [ ]:
ensembl_pathway_relation = '../../usman/CellTICS/reactome/Ensembl2Reactome_All_Levels.txt'
species = 'human'
pathway_names = '../../usman/CellTICS/reactome/ReactomePathways.txt'
pathway_relation = '../../usman/CellTICS/reactome/ReactomePathwaysRelation.txt'
datatype = 'diagnosis'
n_hidden = 3

In [ ]:
pathway_genes = get_gene_pathways(ensembl_pathway_relation, species=species)

In [ ]:
#set(pathway_genes.gene)-set(test_x.index)

In [ ]:
masking, layers_node, train_x, test_x = get_masking(pathway_names,
                                                        pathway_genes,
                                                        pathway_relation,
                                                        train_x,
                                                        test_x,
                                                        train_y,
                                                        datatype,
                                                        species,
                                                        n_hidden)

In [ ]:
train_x

In [ ]:
test_x

In [ ]:
#pd.DataFrame(test_x.index).to_csv('../../preprocessed_data/inhibitory_neuron/marker_genes.csv', index=False)

In [ ]:
train_x.T

In [ ]:
train_y

In [ ]:
diagnosis_mapping = {'AD_with_Plaques': 1, 'NCI_with_No_Plaques': 0}
train_y['diagnosis_binary'] = train_y['diagnosis'].map(diagnosis_mapping)

# Display the first few rows to verify the mapping
print(train_y.head())
train_y.drop(columns=['diagnosis'], inplace=True)
train_y

In [ ]:
train

In [ ]:
qlabel

In [ ]:
try:
    diagnosis_mapping = {'AD_with_Plaques': 1, 'NCI_with_No_Plaques': 0}
    qlabel['diagnosis_binary'] = qlabel['diagnosis'].map(diagnosis_mapping)
    
    # Display the first few rows to verify the mapping
    print(qlabel.head())
    qlabel.drop(columns=['diagnosis'], inplace=True)
    qlabel

except:
    print('Previously Run Before')

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
import pandas as pd

class TabularDataset(Dataset):
    def __init__(self, count_matrix, label):
        # Read the CSV file
        self.data = count_matrix
        # Separate features and target
        self.features = self.data.values
        self.target = label.values
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        # Get features and target for a given index
        features = torch.tensor(self.features[idx], dtype=torch.float32)
        target = torch.tensor(self.target[idx], dtype=torch.float32)
        return features, target

# Example usage



In [ ]:
qlabel

In [ ]:
train_y

In [ ]:
val_x = train_x.T.iloc[40000:,]
val_y = train_y.iloc[40000:,]
train_x = train_x.T.iloc[0:40000,]
train_y = train_y.iloc[0:40000,]


In [ ]:
train_dataset = TabularDataset(train_x,train_y)
val_dataset = TabularDataset(val_x,val_y)
test_dataset = TabularDataset(test_x.T,qlabel)

In [ ]:
val_x

In [ ]:
qlabel

In [ ]:
masking[0].shape, len(layers_node[1])

In [ ]:
masking[1].shape, len(layers_node[2])

In [ ]:
masking[2].shape, len(layers_node[3])

In [ ]:
masking = list(masking.values())
layers_node = list(layers_node.values())

In [ ]:
 import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F


#for output_layer in range(2, len(masking) + 2):       -->n
class CustomNetwork(nn.Module):
    def __init__(self, layers_node, n, masking):
        super(CustomNetwork, self).__init__()
        self.layers = nn.ModuleList()
        self.masks = []
        self.gamma = 0.0001

        for i in range(n-1):
            #print(-1-i)
            #print(layers_node[-1-i])
            in_features = len(layers_node[-1-i])
            out_features = len(layers_node[-1-i-1])
            self.layers.append(nn.Linear(in_features, out_features, bias=False))
            self.masks.append(masking[-1-i])

        self.layers.append(nn.Linear(len(layers_node[-n]), len(layers_node[0])-1))
        self.masks = [torch.tensor(mask, dtype=torch.float32) for mask in self.masks]


    def forward(self, x):
        for i, layer in enumerate(self.layers[:-1]):
            mask = self.masks[i]
            mask = mask.unsqueeze(0).expand(x.size(0), -1, -1)
            #print(mask[0].shape)
            
            #print(layer(x).shape)
            x = layer(x)
            
            '''
            x = x.unsqueeze(2)
            print('x shape before multi: ',x.shape)
            print('shape of mask: ', mask.shape )
            x =  x*mask
            x = x.squeeze(2)
            x = torch.tanh(x)
            '''
            x = torch.relu(x)
            #print('x shape: ',x.shape)
        #print(x.shape)
        
        x = self.layers[-1](x)
        return x


In [ ]:
def evaluate(model, dataloader):
    model.eval()  # Set the model to evaluation mode
    correct = 0
    total = 0
    predicted_list = []
    labels_list = []
    
    with torch.no_grad():  # No need to compute gradients during evaluation
        for features, labels in dataloader:
            outputs = model(features)
            #print(outputs)
            predicted = torch.round(torch.sigmoid(outputs.data))
            #print(outputs)
            #print(predicted)
            #_, predicted = torch.sigmoid(outputs.data)
            predicted_list.append(predicted)
            labels_list.append(labels)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    #print(total)
    accuracy = 100 * correct / total
    return accuracy, predicted_list, labels_list

In [ ]:
from tqdm import tqdm
def model(train_dataloader , test_dataloader, layers_node, masking, output_layer, learning_rate=0.001, num_epochs=50):

    model = CustomNetwork(layers_node, output_layer, masking)
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)  # Using SGD with momentum
    criterion = nn.BCEWithLogitsLoss()

    for epoch in tqdm(range(num_epochs)):
        epoch_cost = 0.
        
        for batch_features,batch_targets in train_dataloader:
            outputs = model(batch_features)
            #print(outputs)
            #print(batch_targets)
            #print(outputs)
            loss = criterion(outputs, batch_targets)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
        
        train_accuracy, predicted_list_train, labels_list_train = evaluate(model, train_dataloader)
        test_accuracy, predicted_list_test, labels_list_test = evaluate(model, test_dataloader)
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}, Train_accuracy: {train_accuracy}, Test_accuracy: {test_accuracy}')
        
    train_accuracy, predicted_list_train, labels_list_train = evaluate(model, train_dataloader)
    test_accuracy, predicted_list_test, labels_list_test = evaluate(model, test_dataloader)
    
    output_train = (predicted_list_train, labels_list_train)
    output_test = (predicted_list_test, labels_list_test)
    
    return output_train, output_test,model

In [ ]:
torch.round(torch.sigmoid(torch.tensor([0.4,0.8])))

In [ ]:

#from neural_network import *
print_information = True
num_epochs = 100
learning_rate = 0.0001
batch_size = 2048
l2_regularization = 0.00001
print_cost = True


# Create the DataLoader
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle= True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle= True)
# Example of iterating through the DataLoader


pred_y_df = pd.DataFrame(data=0, index=test_x.columns, columns=list(range(2, len(masking) + 2)))
train_y_df = pd.DataFrame(data=0, index=train_x.columns, columns=list(range(2, len(masking) + 2)))
#masking = list(masking.values())
model_dict = dict()
#layers_node = list(layers_node.values())
activation_output = {}
for output_layer in range(2, len(masking) + 2):
    if print_information:
        print("Current sub-neural network has " + str(output_layer - 1) + " hidden layers.")
    output_train, output_test,model_dict[output_layer] = model(train_dataloader,
                                          val_dataloader,
                                          layers_node,
                                          masking,
                                          output_layer,
                                          learning_rate=learning_rate,num_epochs=num_epochs
                                          )

In [ ]:
print(model_dict)

In [ ]:
!pip install wandb -Uq

In [203]:
pred_y_df

,2,3,4
AGCTTCCGTATGCTTG.8.11,0,0,0
CATCCGTCATATACCG.10.9,0,0,0
CCTCCTCTCGGTAGAG.32.11,0,0,0
TGTACAGGTGATACAA.26.3,0,0,0
ACATGCAAGGTTAAAC.13.2,0,0,0
...,...,...,...
GATGAGGCACATGTTG.10.2,0,0,0
GGGCGTTCACGTACTA.32.11,0,0,0
AGGGTGAGTGTTACTG.26.3,0,0,0
GACGTTATCGCGTTTC.38.1,0,0,0


In [97]:
'''
    train_y_df[output_layer] = get_prediction(output_train[output_layer],
                                                 pd.get_dummies(train_y['diagnosis']),
                                                 train_x,
                                                 datatype='diagnosis')

    
    for j in range(len(output_train)):
        ctp_sort = layers_node[0]
        ctp_sort.sort()
        if j != output_layer - 1:
            output_train[j + 1] = pd.DataFrame(data=output_train[j + 1],
                                                   index=layers_node[len(layers_node) - 2 - j],
                                                   columns=train_x.columns)
        else:
            output_train[j + 1] = pd.DataFrame(data=output_train[j + 1],
                                                   index=ctp_sort,
                                                   columns=train_x.columns)
    activation_output[output_layer] = output_train
    pred_y_df[output_layer] = get_prediction(output_test[output_layer],
                                                 pd.get_dummies(train_y['diagnosis']),
                                                 test_x,
                                                 datatype='diagnosis')
    
pred_test_y = pd.DataFra000me(data=0, index=test_x.columns, columns=['diagnosis'])
pred_test_y['diagnosis'] = pred_y_df.T.describe().T['top']

pred_train_y = pd.DataFrame(data=0, index= train_x.columns, columns=['diagnosis'])
pred_train_y['diagnosis'] = train_y_df.T.describe().T['top']'''

"\n    train_y_df[output_layer] = get_prediction(output_train[output_layer],\n                                                 pd.get_dummies(train_y['diagnosis']),\n                                                 train_x,\n                                                 datatype='diagnosis')\n\n    \n    for j in range(len(output_train)):\n        ctp_sort = layers_node[0]\n        ctp_sort.sort()\n        if j != output_layer - 1:\n            output_train[j + 1] = pd.DataFrame(data=output_train[j + 1],\n                                                   index=layers_node[len(layers_node) - 2 - j],\n                                                   columns=train_x.columns)\n        else:\n            output_train[j + 1] = pd.DataFrame(data=output_train[j + 1],\n                                                   index=ctp_sort,\n                                                   columns=train_x.columns)\n    activation_output[output_layer] = output_train\n    pred_y_df[output_layer

In [251]:
train_y

,diagnosis
AAGCCATCAGTTAGGG.3.9,NCI_with_No_Plaques
GAACTGTTCTTTGCTA.5.2,NCI_with_No_Plaques
CTAACCCTCTGCGAGC.8.7,NCI_with_No_Plaques
CTACATTGTGTTACAC.24.2,NCI_with_No_Plaques
TTGGTTTCAACAGAGC.1.6,AD_with_Plaques
...,...
GCAGGCTGTCGGCTAC.28.6,NCI_with_No_Plaques
TATCGCCAGCAGATAT.30.10,AD_with_Plaques
ACTATCTTCACGTAGT.31.6,NCI_with_No_Plaques
ATTCAGGCACTTGAGT.27.3,AD_with_Plaques


In [193]:
import torch

# Example tensors
x = torch.randn(1024, 216, 1)  # Shape: [1024, 216, 1]
mask = torch.randn(1024, 216, 159)  # Shape: [1024, 216, 159]

# Perform element-wise multiplication
result = mask * x  # Shape: [1024, 216, 159]

# Print shapes
print("x shape:", x.shape)
print("mask shape:", mask.shape)
print("result shape:", result.shape)



x shape: torch.Size([1024, 216, 1])
mask shape: torch.Size([1024, 216, 159])
result shape: torch.Size([1024, 216, 159])


In [132]:
labels = torch.randint(0, 2, (100,))  
labels

tensor([0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0,
        0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0,
        1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1,
        0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0,
        0, 1, 1, 0])

In [133]:
mask.shape

NameError: name 'mask' is not defined

In [180]:
result

NameError: name 'result' is not defined

In [108]:
import torch
import torch.nn as nn

# Define a simple linear layer
layer = nn.Linear(3, 2)  # Linear layer with 3 input features and 2 output features

# Input tensor
x = torch.tensor([[1.0, 2.0, 3.0]])

# Apply the layer
layer_output = layer(x)

# Apply the tanh activation function
tanh_output = torch.tanh(layer_output)

print("Layer output:", layer_output)
print("After tanh activation:", tanh_output)

Layer output: tensor([[1.8384, 0.1309]], grad_fn=<AddmmBackward0>)
After tanh activation: tensor([[0.9506, 0.1302]], grad_fn=<TanhBackward0>)


In [110]:
layer_output.shape

torch.Size([1, 2])

Current sub-neural network has 1 hidden layers.


KeyError: -1

In [ ]:
sum(pred_train_y.diagnosis.values == rlabel.diagnosis.values)/len(pred_train_y)

In [ ]:
sum(pred_test_y.diagnosis.values == qlabel.diagnosis.values)/len(pred_test_y )

In [52]:
qlabel

,diagnosis
0,AD_with_Plaques
1,AD_with_Plaques
2,AD_with_Plaques
3,AD_with_Plaques
4,NCI_with_No_Plaques
...,...
6879,AD_with_Plaques
6880,NCI_with_No_Plaques
6881,AD_with_Plaques
6882,AD_with_Plaques


In [196]:
import torch
import torch.nn as nn

# Define a simple linear layer
layer = nn.Linear(3, 2)  # Linear layer with 3 input features and 2 output features

# Input tensor
x = torch.tensor([[1.0, 2.0, 3.0]])

# Apply the layer
layer_output = layer(x)

# Apply the tanh activation function
tanh_output = torch.tanh(layer_output)

print("Layer output:", layer_output)
print("After tanh activation:", tanh_output)

Layer output: tensor([[ 1.1401, -0.1478]], grad_fn=<AddmmBackward0>)
After tanh activation: tensor([[ 0.8144, -0.1467]], grad_fn=<TanhBackward0>)


In [197]:
layer_output

tensor([[ 1.1401, -0.1478]], grad_fn=<AddmmBackward0>)

In [198]:
layer.weight

Parameter containing:
tensor([[-0.1207,  0.5761, -0.0801],
        [-0.3824,  0.5135, -0.1200]], requires_grad=True)

In [199]:
layer.weight.shape

torch.Size([2, 3])

In [201]:
x.shape

torch.Size([1, 3])

In [202]:
-0.1207*1 + 0.5761*2 -0.0801*3

0.7911999999999999

In [ ]:
-0.1207,  0.5761, -0.0801

In [204]:
(-0.3824*1) +  (0.5135*2) -(0.1200*3)

0.28459999999999985

In [222]:
import torch
import torch.nn as nn

# Define a simple linear layer
layer = nn.Linear(3, 2, bias = False)  # Linear layer with 3 input features and 2 output features

# Input tensor
x = torch.tensor([[1.0, 2.0, 3.0]])

# Apply the layer
layer_output = layer(x)

# Apply the tanh activation function
tanh_output = torch.tanh(layer_output)

print("Layer output:", layer_output)
print("After tanh activation:", tanh_output)

Layer output: tensor([[-0.7807,  0.0361]], grad_fn=<MmBackward0>)
After tanh activation: tensor([[-0.6531,  0.0361]], grad_fn=<TanhBackward0>)


In [223]:
layer.weight.shape

torch.Size([2, 3])

In [224]:
x.shape

torch.Size([1, 3])

In [229]:
# What nn.linear does

torch.matmul(x,layer.weight.T)    


tensor([[-0.7807,  0.0361]], grad_fn=<MmBackward0>)

In [226]:
layer.weight

Parameter containing:
tensor([[ 0.4250,  0.2475, -0.5669],
        [-0.4622,  0.5361, -0.1913]], requires_grad=True)

In [227]:
1*0.5236 -0.5384*2-0.4127*3

-1.7913000000000001

In [228]:
layer_output

tensor([[-0.7807,  0.0361]], grad_fn=<MmBackward0>)

In [230]:
import torch

# Create a batch of tensors
num_samples = 1024
num_features = 3
x_batch = torch.randn(num_samples, num_features)  # Shape: [1024, 3]§

# Print the shape and content of the batch
print("x_batch shape:", x_batch.shape)
print("x_batch tensor:", x_batch)


x_batch shape: torch.Size([1024, 3])
x_batch tensor: tensor([[ 1.3851,  0.2620, -1.0035],
        [-0.3788,  1.1610, -1.9341],
        [ 0.1912,  1.6240, -1.6308],
        ...,
        [ 0.0301, -0.4233,  1.8012],
        [ 1.7705,  0.0975, -0.7867],
        [ 1.1085,  0.7245, -0.5201]])


In [231]:
layer = nn.Linear(3, 2, bias = False) 

In [234]:
layer.weight

Parameter containing:
tensor([[-0.1641,  0.5494,  0.1831],
        [ 0.2614, -0.2852, -0.3680]], requires_grad=True)

In [232]:
layer_output = layer(x_batch)

In [233]:
layer_output 

tensor([[-0.2671,  0.6567],
        [ 0.3457,  0.2817],
        [ 0.5621,  0.1870],
        ...,
        [ 0.0924, -0.5343],
        [-0.3810,  0.7246],
        [ 0.1209,  0.2746]], grad_fn=<MmBackward0>)

In [236]:
1.3851 * -0.1641 + 0.2640 * 0.5494 + 0.1831 * -1.0035

-0.26599416

# Understading Masking and batch intorch
